In [ ]:
pip install contractions
!pip install fastText

     |████████████████████████████████| 287 kB 4.0 MB/s 
     |████████████████████████████████| 106 kB 49.7 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import nltk
import string
# !pip install fastText
import fasttext
import contractions
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
# plt.xticks(rotation=70)
# pd.options.mode.chained_assignment = None
# pd.set_option('display.max_colwidth', 100)
# %matplotlib inline

In [ ]:
with open('Tweets.csv') as f:
    df = pd.read_csv(f)
f.close()

In [ ]:
df.head

<bound method NDFrame.head of                  tweet_id airline_sentiment  airline_sentiment_confidence  \
0      570306133677760513           neutral                        1.0000   
1      570301130888122368          positive                        0.3486   
2      570301083672813571           neutral                        0.6837   
3      570301031407624196          negative                        1.0000   
4      570300817074462722          negative                        1.0000   
...                   ...               ...                           ...   
14635  569587686496825344          positive                        0.3487   
14636  569587371693355008          negative                        1.0000   
14637  569587242672398336           neutral                        1.0000   
14638  569587188687634433          negative                        1.0000   
14639  569587140490866689           neutral                        0.6771   

               negativereason  negativereason

In [ ]:
for col in df.columns:
    print(col, df[col].isnull().sum())


tweet_id 0
airline_sentiment 0
airline_sentiment_confidence 0
negativereason 5462
negativereason_confidence 4118
airline 0
airline_sentiment_gold 14600
name 0
negativereason_gold 14608
retweet_count 0
text 0
tweet_coord 13621
tweet_created 0
tweet_location 4733
user_timezone 4820


In [ ]:
rws = df.loc[:, ['text']]
rws

,text
0,@VirginAmerica What @dhepburn said.
1,@VirginAmerica plus you've added commercials t...
2,@VirginAmerica I didn't today... Must mean I n...
3,@VirginAmerica it's really aggressive to blast...
4,@VirginAmerica and it's a really big bad thing...
...,...
14635,@AmericanAir thank you we got on a different f...
14636,@AmericanAir leaving over 20 minutes Late Flig...
14637,@AmericanAir Please bring American Airlines to...
14638,"@AmericanAir you have my money, you change my ..."


In [ ]:
# Expanding Contractions
rws['no_contract'] = rws['text'].apply(lambda x: [contractions.fix(word) for word in x.split()])
rws.head()

,text,no_contract
0,@VirginAmerica What @dhepburn said.,"[@VirginAmerica, What, @dhepburn, said.]"
1,@VirginAmerica plus you've added commercials t...,"[@VirginAmerica, plus, you have, added, commer..."
2,@VirginAmerica I didn't today... Must mean I n...,"[@VirginAmerica, I, did not, today..., Must, m..."
3,@VirginAmerica it's really aggressive to blast...,"[@VirginAmerica, it is, really, aggressive, to..."
4,@VirginAmerica and it's a really big bad thing...,"[@VirginAmerica, and, it is, a, really, big, b..."


In [ ]:
rws['text_str'] = [' '.join(map(str, l)) for l in rws['no_contract']]
rws.head()


,text,no_contract,text_str
0,@VirginAmerica What @dhepburn said.,"[@VirginAmerica, What, @dhepburn, said.]",@VirginAmerica What @dhepburn said.
1,@VirginAmerica plus you've added commercials t...,"[@VirginAmerica, plus, you have, added, commer...",@VirginAmerica plus you have added commercials...
2,@VirginAmerica I didn't today... Must mean I n...,"[@VirginAmerica, I, did not, today..., Must, m...",@VirginAmerica I did not today... Must mean I ...
3,@VirginAmerica it's really aggressive to blast...,"[@VirginAmerica, it is, really, aggressive, to...",@VirginAmerica it is really aggressive to blas...
4,@VirginAmerica and it's a really big bad thing...,"[@VirginAmerica, and, it is, a, really, big, b...",@VirginAmerica and it is a really big bad thin...


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
rws['tokenized'] = rws['text_str'].apply(word_tokenize)
rws.head()

,text,no_contract,text_str,tokenized
0,@VirginAmerica What @dhepburn said.,"[@VirginAmerica, What, @dhepburn, said.]",@VirginAmerica What @dhepburn said.,"[@, VirginAmerica, What, @, dhepburn, said, .]"
1,@VirginAmerica plus you've added commercials t...,"[@VirginAmerica, plus, you have, added, commer...",@VirginAmerica plus you have added commercials...,"[@, VirginAmerica, plus, you, have, added, com..."
2,@VirginAmerica I didn't today... Must mean I n...,"[@VirginAmerica, I, did not, today..., Must, m...",@VirginAmerica I did not today... Must mean I ...,"[@, VirginAmerica, I, did, not, today, ..., Mu..."
3,@VirginAmerica it's really aggressive to blast...,"[@VirginAmerica, it is, really, aggressive, to...",@VirginAmerica it is really aggressive to blas...,"[@, VirginAmerica, it, is, really, aggressive,..."
4,@VirginAmerica and it's a really big bad thing...,"[@VirginAmerica, and, it is, a, really, big, b...",@VirginAmerica and it is a really big bad thin...,"[@, VirginAmerica, and, it, is, a, really, big..."


In [ ]:
rws['lower'] = rws['tokenized'].apply(lambda x: [word.lower() for word in x])
rws.head()

,text,no_contract,text_str,tokenized,lower
0,@VirginAmerica What @dhepburn said.,"[@VirginAmerica, What, @dhepburn, said.]",@VirginAmerica What @dhepburn said.,"[@, VirginAmerica, What, @, dhepburn, said, .]","[@, virginamerica, what, @, dhepburn, said, .]"
1,@VirginAmerica plus you've added commercials t...,"[@VirginAmerica, plus, you have, added, commer...",@VirginAmerica plus you have added commercials...,"[@, VirginAmerica, plus, you, have, added, com...","[@, virginamerica, plus, you, have, added, com..."
2,@VirginAmerica I didn't today... Must mean I n...,"[@VirginAmerica, I, did not, today..., Must, m...",@VirginAmerica I did not today... Must mean I ...,"[@, VirginAmerica, I, did, not, today, ..., Mu...","[@, virginamerica, i, did, not, today, ..., mu..."
3,@VirginAmerica it's really aggressive to blast...,"[@VirginAmerica, it is, really, aggressive, to...",@VirginAmerica it is really aggressive to blas...,"[@, VirginAmerica, it, is, really, aggressive,...","[@, virginamerica, it, is, really, aggressive,..."
4,@VirginAmerica and it's a really big bad thing...,"[@VirginAmerica, and, it is, a, really, big, b...",@VirginAmerica and it is a really big bad thin...,"[@, VirginAmerica, and, it, is, a, really, big...","[@, virginamerica, and, it, is, a, really, big..."


In [ ]:
punc = string.punctuation
rws['no_punc'] = rws['lower'].apply(lambda x: [word for word in x if word not in punc])
rws.head()

,text,no_contract,text_str,tokenized,lower,no_punc
0,@VirginAmerica What @dhepburn said.,"[@VirginAmerica, What, @dhepburn, said.]",@VirginAmerica What @dhepburn said.,"[@, VirginAmerica, What, @, dhepburn, said, .]","[@, virginamerica, what, @, dhepburn, said, .]","[virginamerica, what, dhepburn, said]"
1,@VirginAmerica plus you've added commercials t...,"[@VirginAmerica, plus, you have, added, commer...",@VirginAmerica plus you have added commercials...,"[@, VirginAmerica, plus, you, have, added, com...","[@, virginamerica, plus, you, have, added, com...","[virginamerica, plus, you, have, added, commer..."
2,@VirginAmerica I didn't today... Must mean I n...,"[@VirginAmerica, I, did not, today..., Must, m...",@VirginAmerica I did not today... Must mean I ...,"[@, VirginAmerica, I, did, not, today, ..., Mu...","[@, virginamerica, i, did, not, today, ..., mu...","[virginamerica, i, did, not, today, ..., must,..."
3,@VirginAmerica it's really aggressive to blast...,"[@VirginAmerica, it is, really, aggressive, to...",@VirginAmerica it is really aggressive to blas...,"[@, VirginAmerica, it, is, really, aggressive,...","[@, virginamerica, it, is, really, aggressive,...","[virginamerica, it, is, really, aggressive, to..."
4,@VirginAmerica and it's a really big bad thing...,"[@VirginAmerica, and, it is, a, really, big, b...",@VirginAmerica and it is a really big bad thin...,"[@, VirginAmerica, and, it, is, a, really, big...","[@, virginamerica, and, it, is, a, really, big...","[virginamerica, and, it, is, a, really, big, b..."


In [ ]:
stop_words = set(stopwords.words('english'))
rws['stopwords_removed'] = rws['no_punc'].apply(lambda x: [word for word in x if word not in stop_words])
rws.head()


,text,no_contract,text_str,tokenized,lower,no_punc,stopwords_removed
0,@VirginAmerica What @dhepburn said.,"[@VirginAmerica, What, @dhepburn, said.]",@VirginAmerica What @dhepburn said.,"[@, VirginAmerica, What, @, dhepburn, said, .]","[@, virginamerica, what, @, dhepburn, said, .]","[virginamerica, what, dhepburn, said]","[virginamerica, dhepburn, said]"
1,@VirginAmerica plus you've added commercials t...,"[@VirginAmerica, plus, you have, added, commer...",@VirginAmerica plus you have added commercials...,"[@, VirginAmerica, plus, you, have, added, com...","[@, virginamerica, plus, you, have, added, com...","[virginamerica, plus, you, have, added, commer...","[virginamerica, plus, added, commercials, expe..."
2,@VirginAmerica I didn't today... Must mean I n...,"[@VirginAmerica, I, did not, today..., Must, m...",@VirginAmerica I did not today... Must mean I ...,"[@, VirginAmerica, I, did, not, today, ..., Mu...","[@, virginamerica, i, did, not, today, ..., mu...","[virginamerica, i, did, not, today, ..., must,...","[virginamerica, today, ..., must, mean, need, ..."
3,@VirginAmerica it's really aggressive to blast...,"[@VirginAmerica, it is, really, aggressive, to...",@VirginAmerica it is really aggressive to blas...,"[@, VirginAmerica, it, is, really, aggressive,...","[@, virginamerica, it, is, really, aggressive,...","[virginamerica, it, is, really, aggressive, to...","[virginamerica, really, aggressive, blast, obn..."
4,@VirginAmerica and it's a really big bad thing...,"[@VirginAmerica, and, it is, a, really, big, b...",@VirginAmerica and it is a really big bad thin...,"[@, VirginAmerica, and, it, is, a, really, big...","[@, virginamerica, and, it, is, a, really, big...","[virginamerica, and, it, is, a, really, big, b...","[virginamerica, really, big, bad, thing]"


In [ ]:
rws['pos_tags'] = rws['stopwords_removed'].apply(nltk.tag.pos_tag)
rws.head()

,text,no_contract,text_str,tokenized,lower,no_punc,stopwords_removed,pos_tags
0,@VirginAmerica What @dhepburn said.,"[@VirginAmerica, What, @dhepburn, said.]",@VirginAmerica What @dhepburn said.,"[@, VirginAmerica, What, @, dhepburn, said, .]","[@, virginamerica, what, @, dhepburn, said, .]","[virginamerica, what, dhepburn, said]","[virginamerica, dhepburn, said]","[(virginamerica, NN), (dhepburn, NN), (said, V..."
1,@VirginAmerica plus you've added commercials t...,"[@VirginAmerica, plus, you have, added, commer...",@VirginAmerica plus you have added commercials...,"[@, VirginAmerica, plus, you, have, added, com...","[@, virginamerica, plus, you, have, added, com...","[virginamerica, plus, you, have, added, commer...","[virginamerica, plus, added, commercials, expe...","[(virginamerica, NN), (plus, CC), (added, JJ),..."
2,@VirginAmerica I didn't today... Must mean I n...,"[@VirginAmerica, I, did not, today..., Must, m...",@VirginAmerica I did not today... Must mean I ...,"[@, VirginAmerica, I, did, not, today, ..., Mu...","[@, virginamerica, i, did, not, today, ..., mu...","[virginamerica, i, did, not, today, ..., must,...","[virginamerica, today, ..., must, mean, need, ...","[(virginamerica, NN), (today, NN), (..., :), (..."
3,@VirginAmerica it's really aggressive to blast...,"[@VirginAmerica, it is, really, aggressive, to...",@VirginAmerica it is really aggressive to blas...,"[@, VirginAmerica, it, is, really, aggressive,...","[@, virginamerica, it, is, really, aggressive,...","[virginamerica, it, is, really, aggressive, to...","[virginamerica, really, aggressive, blast, obn...","[(virginamerica, NNS), (really, RB), (aggressi..."
4,@VirginAmerica and it's a really big bad thing...,"[@VirginAmerica, and, it is, a, really, big, b...",@VirginAmerica and it is a really big bad thin...,"[@, VirginAmerica, and, it, is, a, really, big...","[@, virginamerica, and, it, is, a, really, big...","[virginamerica, and, it, is, a, really, big, b...","[virginamerica, really, big, bad, thing]","[(virginamerica, NNS), (really, RB), (big, JJ)..."


In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
rws['wordnet_pos'] = rws['pos_tags'].apply(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])
rws.head()

,text,no_contract,text_str,tokenized,lower,no_punc,stopwords_removed,pos_tags,wordnet_pos
0,@VirginAmerica What @dhepburn said.,"[@VirginAmerica, What, @dhepburn, said.]",@VirginAmerica What @dhepburn said.,"[@, VirginAmerica, What, @, dhepburn, said, .]","[@, virginamerica, what, @, dhepburn, said, .]","[virginamerica, what, dhepburn, said]","[virginamerica, dhepburn, said]","[(virginamerica, NN), (dhepburn, NN), (said, V...","[(virginamerica, n), (dhepburn, n), (said, v)]"
1,@VirginAmerica plus you've added commercials t...,"[@VirginAmerica, plus, you have, added, commer...",@VirginAmerica plus you have added commercials...,"[@, VirginAmerica, plus, you, have, added, com...","[@, virginamerica, plus, you, have, added, com...","[virginamerica, plus, you, have, added, commer...","[virginamerica, plus, added, commercials, expe...","[(virginamerica, NN), (plus, CC), (added, JJ),...","[(virginamerica, n), (plus, n), (added, a), (c..."
2,@VirginAmerica I didn't today... Must mean I n...,"[@VirginAmerica, I, did not, today..., Must, m...",@VirginAmerica I did not today... Must mean I ...,"[@, VirginAmerica, I, did, not, today, ..., Mu...","[@, virginamerica, i, did, not, today, ..., mu...","[virginamerica, i, did, not, today, ..., must,...","[virginamerica, today, ..., must, mean, need, ...","[(virginamerica, NN), (today, NN), (..., :), (...","[(virginamerica, n), (today, n), (..., n), (mu..."
3,@VirginAmerica it's really aggressive to blast...,"[@VirginAmerica, it is, really, aggressive, to...",@VirginAmerica it is really aggressive to blas...,"[@, VirginAmerica, it, is, really, aggressive,...","[@, virginamerica, it, is, really, aggressive,...","[virginamerica, it, is, really, aggressive, to...","[virginamerica, really, aggressive, blast, obn...","[(virginamerica, NNS), (really, RB), (aggressi...","[(virginamerica, n), (really, r), (aggressive,..."
4,@VirginAmerica and it's a really big bad thing...,"[@VirginAmerica, and, it is, a, really, big, b...",@VirginAmerica and it is a really big bad thin...,"[@, VirginAmerica, and, it, is, a, really, big...","[@, virginamerica, and, it, is, a, really, big...","[virginamerica, and, it, is, a, really, big, b...","[virginamerica, really, big, bad, thing]","[(virginamerica, NNS), (really, RB), (big, JJ)...","[(virginamerica, n), (really, r), (big, a), (b..."


In [ ]:
wnl = WordNetLemmatizer()
rws['lemmatized'] = rws['wordnet_pos'].apply(lambda x: [wnl.lemmatize(word, tag) for word, tag in x])
rws.head()

,text,no_contract,text_str,tokenized,lower,no_punc,stopwords_removed,pos_tags,wordnet_pos,lemmatized
0,@VirginAmerica What @dhepburn said.,"[@VirginAmerica, What, @dhepburn, said.]",@VirginAmerica What @dhepburn said.,"[@, VirginAmerica, What, @, dhepburn, said, .]","[@, virginamerica, what, @, dhepburn, said, .]","[virginamerica, what, dhepburn, said]","[virginamerica, dhepburn, said]","[(virginamerica, NN), (dhepburn, NN), (said, V...","[(virginamerica, n), (dhepburn, n), (said, v)]","[virginamerica, dhepburn, say]"
1,@VirginAmerica plus you've added commercials t...,"[@VirginAmerica, plus, you have, added, commer...",@VirginAmerica plus you have added commercials...,"[@, VirginAmerica, plus, you, have, added, com...","[@, virginamerica, plus, you, have, added, com...","[virginamerica, plus, you, have, added, commer...","[virginamerica, plus, added, commercials, expe...","[(virginamerica, NN), (plus, CC), (added, JJ),...","[(virginamerica, n), (plus, n), (added, a), (c...","[virginamerica, plus, added, commercial, exper..."
2,@VirginAmerica I didn't today... Must mean I n...,"[@VirginAmerica, I, did not, today..., Must, m...",@VirginAmerica I did not today... Must mean I ...,"[@, VirginAmerica, I, did, not, today, ..., Mu...","[@, virginamerica, i, did, not, today, ..., mu...","[virginamerica, i, did, not, today, ..., must,...","[virginamerica, today, ..., must, mean, need, ...","[(virginamerica, NN), (today, NN), (..., :), (...","[(virginamerica, n), (today, n), (..., n), (mu...","[virginamerica, today, ..., must, mean, need, ..."
3,@VirginAmerica it's really aggressive to blast...,"[@VirginAmerica, it is, really, aggressive, to...",@VirginAmerica it is really aggressive to blas...,"[@, VirginAmerica, it, is, really, aggressive,...","[@, virginamerica, it, is, really, aggressive,...","[virginamerica, it, is, really, aggressive, to...","[virginamerica, really, aggressive, blast, obn...","[(virginamerica, NNS), (really, RB), (aggressi...","[(virginamerica, n), (really, r), (aggressive,...","[virginamerica, really, aggressive, blast, obn..."
4,@VirginAmerica and it's a really big bad thing...,"[@VirginAmerica, and, it is, a, really, big, b...",@VirginAmerica and it is a really big bad thin...,"[@, VirginAmerica, and, it, is, a, really, big...","[@, virginamerica, and, it, is, a, really, big...","[virginamerica, and, it, is, a, really, big, b...","[virginamerica, really, big, bad, thing]","[(virginamerica, NNS), (really, RB), (big, JJ)...","[(virginamerica, n), (really, r), (big, a), (b...","[virginamerica, really, big, bad, thing]"


In [ ]:
rws.to_csv('clean.csv')